### Корректное разбитие на train/test

In [8]:
from collections import Counter
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter, defaultdict
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.utils import shuffle
import tqdm

%matplotlib inline

import sys

import pickle

In [2]:
N_JOBS = 32
random_seed = 42
alpha = 0.1
MAX_FEATURES = 300

data_path = '/mnt/dump/df.xlsx'

train_path = '/mnt/dump/df_train.xlsx'
test_path = '/mnt/dump/df_test.xlsx'

In [3]:
data = pd.read_excel(data_path)

In [4]:
data.head()

,label,text,title
0,14,convent intern commiss rhine bern convent le g...,environmental_protection
1,14,convent intern commiss rhine bern convent le g...,pollution_control_measures
2,14,convent intern commiss rhine bern convent le g...,environmental_cooperation
3,14,convent intern commiss rhine bern convent le g...,rhine_valley
4,14,convent intern commiss rhine bern convent le g...,ec_agreement


In [5]:
data['text'].nunique(), data['label'].nunique()

(19281, 19314)

Поделим выборку на train и test 1 к 9

In [9]:
labels = list(data['label'].unique())
labels = shuffle(labels, random_state=random_seed)

N = int(len(labels)*alpha)
label_train = labels[:-N]
label_test  = labels[-N:]

In [114]:
df_train = data[data['label'].isin(label_train)]
df_test  = data[data['label'].isin(label_test)]

df_train.head()

In [116]:
len(set(df_test['title'])-set(df_train['title'])), \
len(set(df_train['title'])-set(df_test['title']))

(85, 1864)

Этот вариент не корректный, так как иногда некоторые классы встречаются только в train или в test

удалим такие классы

In [117]:
titles = list(set(df_test['title']) & set(df_train['title']))
              
df_test = df_test[df_test['title'].isin(titles)]
df_train = df_train[df_train['title'].isin(titles)]

In [124]:
print('title test  ', df_test['title'].nunique(), '\ntitle train ', df_train['title'].nunique())
print('labels train&test', len(set(df_train['label'].unique()) & set(df_test['label'].unique())))

title test   2007 
title train  2007
labels train&test 0


In [127]:
df_train.to_excel(train_path)
df_test.to_excel(test_path)

In [7]:
df_train.shape, df_test.shape

((85219, 3), (10229, 3))

In [303]:
# del data_train, data_test

In [30]:
from sklearn.preprocessing import LabelEncoder
lbr = LabelEncoder()
df_train['title'] = lbr.fit_transform(df_train['title'])
df_test['title']  = lbr.transform(df_test['title'])

In [49]:
# lbr.inverse_transform(df_train['title'])
pickle.dump(lbr, open('/mnt/dump/data/label_encoder.pickle', 'wb'))

max_iter reached after 1998 seconds
max_iter reached after 2003 seconds
max_iter reached after 2004 seconds
max_iter reached after 2005 seconds
max_iter reached after 2007 seconds
max_iter reached after 2007 seconds
max_iter reached after 2008 seconds
max_iter reached after 2008 seconds
max_iter reached after 2009 seconds
max_iter reached after 2010 seconds
max_iter reached after 2011 seconds
max_iter reached after 2012 seconds
max_iter reached after 2012 seconds
max_iter reached after 2013 seconds
max_iter reached after 2014 seconds
max_iter reached after 2014 seconds
max_iter reached after 2014 seconds
max_iter reached after 2015 seconds
max_iter reached after 2018 seconds
max_iter reached after 2018 seconds
max_iter reached after 2019 seconds
max_iter reached after 2020 seconds
max_iter reached after 2021 seconds
max_iter reached after 2021 seconds
max_iter reached after 2021 seconds
max_iter reached after 2021 seconds
max_iter reached after 2021 seconds
max_iter reached after 2021 

Загрузка данных

In [10]:
df_train = pd.read_excel(train_path)
print('loaded')

In [11]:
df_test = pd.read_excel(test_path)
print('loaded')

### Подготовим ембединги

#### CountVectorizer

In [11]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
vect_count = CountVectorizer(max_features = MAX_FEATURES)
vect_count.fit(data['text'])

CPU times: user 59.4 s, sys: 503 ms, total: 59.9 s
Wall time: 59.9 s


In [12]:
%%time
M_count_vect_train = vect_count.transform(df_train['text'])
M_count_vect_test  = vect_count.transform(df_test['text'])


pickle.dump(M_count_vect_train, open('/mnt/dump/dump_pipeline/M_count_vect_train.pickle', 'wb'))
pickle.dump(M_count_vect_test,  open('/mnt/dump/dump_pipeline/M_count_vect_test.pickle', 'wb'))

CPU times: user 58.2 s, sys: 310 ms, total: 58.6 s
Wall time: 58.6 s


In [299]:
# M_count_vect_train.shape, data_train['title'].shape

#### TF-IDF vectorizer

In [13]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect_tf_idf = TfidfVectorizer(max_features = MAX_FEATURES)
vect_tf_idf.fit(data['text'])

CPU times: user 59 s, sys: 487 ms, total: 59.5 s
Wall time: 59.5 s


In [14]:
M_tf_idf_vect_train = vect_tf_idf.transform(df_train['text'])
M_tf_idf_vect_test  = vect_tf_idf.transform(df_test['text'])


pickle.dump(M_tf_idf_vect_train, open('/mnt/dump/dump_pipeline/M_tf_idf_vect_train.pickle', 'wb'))
pickle.dump(M_tf_idf_vect_test,  open('/mnt/dump/dump_pipeline/M_tf_idf_vect_test.pickle', 'wb'))

#### Word2Vec (на нашей выборке)

In [ ]:
# M_tf_idf_vect_test.shape

In [235]:
# text = df_train['text'].get_values()[0]
# vectors, tf_idfs = 0, 0
# emb_tf_idf = vect_tf_idf.transform([text]).toarray()
# words_dict = {v:k for k,v in vect_tf_idf.vocabulary_.items()}
# # words_dict
# final = None
# for line in emb_tf_idf:
#     indexes = np.where(abs(line) >1e-10)[0]
#     for word_ind in indexes:
#         word = words_dict[word_ind]
#         if word in word2vec_our_model.wv:
#             if final is None:
#                 final = word2vec_our_model.wv[word] * line[word_ind]
#     final = final/np.norm(final)
# np.sum(np.abs(final))
# # # for c, word in text.split(' '):
# # #     if word in word2vec_our_model.wv:
# # #         vectors += word2vec_our_model.wv[word]
# # #         tf_idfs = 
# # # word2vec_our_model.wv[text.split(' ')[0]]
# # # text

In [234]:
# vect_tf_idf.transform([text]).toarray()

In [171]:
Word2VecVocabulary

1500

In [15]:
%%time
Word2VecVocabulary = MAX_FEATURES*5
word2vec_our_model = Word2Vec([i.split(' ') for i in tqdm(data['text'])], 
                              max_vocab_size=Word2VecVocabulary, 
                              size=MAX_FEATURES, window=5, min_count=5, workers=32, seed=1)

NameError: name 'tqdm' is not defined

In [ ]:
# a = text.split(' ')[0]

In [ ]:
# a in word2vec_our_model

In [16]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec, max_features = None):
        self.word2vec = word2vec
        self.word2weight = None
#         self.dim = word2vec.# len(word2vec.itervalues().next())
#         if True:#hasattr(self.word2vec., 'corpus_count'):
        self.dim = self.word2vec.wv.vector_size
#         else:
#             if len(word2vec)>0:
#                 self.dim=len(word2vec[next(iter(glove))])
#             else:
#                 self.dim=0
                
    def fit(self, X, y=None, use_default_weights = True):
#         tfidf = TfidfVectorizer(analyzer=lambda x: x)
#         tfidf.fit(X)
        if use_default_weights:
            tfidf = vect_tf_idf
        else:
            tfidf = TfidfVectorizer(analyzer=lambda x: x)
            tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
#         if split:
#             split_func = lambda x: [i.split() for i in x]
#         else:
#             split_func = lambda x: x
            
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tqdm(tfidf.vocabulary_.items())])

        return self

    def transform(self, X, split=True):
        if split:
            split_func = lambda x: [i.split() for i in x]
        else:
            split_func = lambda x: x
        return np.array([
                np.mean([self.word2vec.wv[w] * self.word2weight[w]
                         for w in words if w in self.word2vec.wv] or
                        [np.zeros(self.dim)], axis=0)
                for words in tqdm(split_func(X))
            ])

In [ ]:
# data__splited = [i.split() for i in tqdm(data['text'])]
# data_test_splitted = [i.split() for i in tqdm(data['text'])]

In [17]:
%%time
vect_word2vec_our = TfidfEmbeddingVectorizer(word2vec_our_model, max_features=MAX_FEATURES)
vect_word2vec_our.fit(None)#[i.split() for i in tqdm(data['text'])])

NameError: name 'word2vec_our_model' is not defined

In [269]:
# a = vect_word2vec_our.transform(df_train['text'].get_values()[:10], split=True)

In [270]:
# vect_word2vec_our.word2vec.corpus_count

In [18]:
%%time
M_word2vec_our_vect_train = vect_word2vec_our.transform(df_train['text'], split=True)
M_word2vec_our_vect_test  = vect_word2vec_our.transform(df_test['text'], split=True)


pickle.dump(M_word2vec_our_vect_train, open('/mnt/dump/dump_pipeline/M_word2vec_our_vect_train.pickle', 'wb'))
pickle.dump(M_word2vec_our_vect_test,  open('/mnt/dump/dump_pipeline/M_word2vec_our_vect_test.pickle', 'wb'))

NameError: name 'vect_word2vec_our' is not defined

In [ ]:
# pickle.dump(M_word2vec_our_vect_train, open('/mnt/dump/dump_pipeline/M_word2vec_our_vect_train.pickle', 'wb'))
# pickle.dump(M_word2vec_our_vect_test,  open('/mnt/dump/dump_pipeline/M_word2vec_our_vect_test.pickle', 'wb'))

In [ ]:
# vect_word2vec_our.transform('Пошел на хуй 90')

In [ ]:
# M_word2vec_our_vect_test.shape
# plt.hist([np.sum(np.abs(i)) for i in M_word2vec_our_vect_train])
# plt.show()

#### Word2Vec (обученный на GoogleNews)

In [19]:
%%time
word2vec_google = gensim.models.KeyedVectors.load_word2vec_format('/mnt/dump/data/GoogleNews-vectors-negative300.bin', binary=True)
vect_word2vec_google = TfidfEmbeddingVectorizer(word2vec_google, max_features=MAX_FEATURES)
vect_word2vec_google.fit(data['text'])

/home/autodeploy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


NameError: name 'tqdm' is not defined

In [20]:
%%time
M_word2vec_google_vect_train = vect_word2vec_google.transform(df_train['text'])
M_word2vec_google_vect_test  = vect_word2vec_google.transform(df_test['text'])


pickle.dump(M_word2vec_google_vect_train, open('/mnt/dump/dump_pipeline/M_word2vec_google_vect_train.pickle', 'wb'))
pickle.dump(M_word2vec_google_vect_test,  open('/mnt/dump/dump_pipeline/M_word2vec_google_vect_test.pickle', 'wb'))

NameError: name 'tqdm' is not defined

Doc2Vec

In [182]:
def tag_docs(docs, col): 
    tagged = docs.apply(lambda r:TaggedDocument(words=simple_preprocess(r[col]),tags=[r.title]),  axis=1) 
    return tagged

def train_doc2vec_model(tagged_docs, sents=None):
    if sents is None:
        sents = tagged_docs.values
    doc2vec_model = Doc2Vec(sents, dbow_words=0, workers=30) 
    return doc2vec_model

def vec_for_learning(doc2vec_model, tagged_docs):
    sents = tagged_docs.values 
    targets, regressors = zip([(doc.tags[0], doc2vec_model.infer_vector(doc.words, steps=20)) for doc in tqdm(sents)])
    return targets, regressors

In [203]:
res = {}
for label, obj in df_train.groupby('label'):
    text = obj['text'].get_values()[0]
    res[text] = obj['title'].get_values()

In [201]:
from tqdm import tqdm

In [204]:
all_docs = [TaggedDocument(words=simple_preprocess(k), tags = v) for k, v in tqdm(res.items())]

100%|██████████| 17351/17351 [00:18<00:00, 914.75it/s] 


In [206]:
vect_doc2vec = Doc2Vec(all_docs, dbow_words=0, workers=50)

In [213]:
M_doc2Vec_test = np.array([vect_doc2vec.infer_vector(simple_preprocess(txt)) for txt in tqdm( df_test['text'])])


 32%|███▏      | 3251/10229 [00:25<01:11, 97.93it/s] 


 56%|█████▌    | 5702/10229 [00:49<01:15, 60.00it/s]


 77%|███████▋  | 7924/10229 [01:13<00:29, 78.18it/s]


  0%|          | 101/85219 [00:02<35:18, 40.19it/s]


  3%|▎         | 2201/85219 [00:27<44:08, 31.34it/s]


...

 89%|████████▉ | 75921/85219 [11:39<01:30, 102.39it/s]


 92%|█████████▏| 78418/85219 [12:02<02:09, 52.33it/s]


 95%|█████████▌| 81047/85219 [12:26<01:02, 66.66it/s]


 97%|█████████▋| 83066/85219 [12:49<00:28, 76.79it/s]


100%|██████████| 85219/85219 [13:07<00:00, 108.16it/s]

In [223]:
from copy import deepcopy
говно = {}
def foo(txt):
    if txt in говно:
        return говно[txt]
    else:
        res = vect_doc2vec.infer_vector(simple_preprocess(txt))
        говно[txt] = deepcopy(res)
        return res
    
M_doc2vec_train = np.array([foo(txt) for txt in tqdm(df_train['text'])])






  0%|          | 0/85219 [00:00<?, ?it/s]




  0%|          | 24/85219 [00:00<06:45, 210.09it/s]




  0%|          | 43/85219 [00:00<07:01, 202.05it/s]




  0%|          | 58/85219 [00:00<07:53, 179.92it/s]




  0%|          | 78/85219 [00:00<07:42, 184.19it/s]




  0%|          | 126/85219 [00:00<06:20, 223.49it/s]




  0%|          | 153/85219 [00:00<06:07, 231.48it/s]




  0%|          | 213/85219 [00:00<05:04, 279.55it/s]




  0%|          | 246/85219 [00:00<05:03, 280.29it/s]




  0%|          | 278/85219 [00:00<05:00, 283.10it/s]




  0%|          | 309/85219 [00:01<05:01, 281.99it/s]




  0%|          | 339/85219 [00:01<06:27, 219.24it/s]




  0%|          | 404/85219 [00:01<05:11, 272.67it/s]




  1%|          | 441/85219 [00:01<05:08, 274.53it/s]




  1%|          | 510/85219 [00:01<04:14, 333.36it/s]




  1%|          | 587/85219 [00:01<03:38, 386.68it/s]




  1%|          | 684/85219 [00:01<02:59, 470.43it/s]




  1%|          | 752/85219 [00:01<02:43, 

 15%|█▍        | 12734/85219 [00:18<01:49, 662.52it/s]




 15%|█▌        | 12985/85219 [00:18<01:25, 848.41it/s]




 15%|█▌        | 13208/85219 [00:18<01:09, 1042.08it/s]




 16%|█▌        | 13378/85219 [00:19<01:24, 854.28it/s] 




 16%|█▌        | 13515/85219 [00:19<01:50, 646.28it/s]




 16%|█▌        | 13624/85219 [00:19<02:24, 496.74it/s]




 16%|█▌        | 13710/85219 [00:19<02:16, 524.24it/s]




 16%|█▋        | 13946/85219 [00:19<01:44, 682.24it/s]




 17%|█▋        | 14266/85219 [00:20<01:19, 892.43it/s]




 17%|█▋        | 14537/85219 [00:20<01:03, 1116.13it/s]




 17%|█▋        | 14745/85219 [00:20<00:59, 1193.09it/s]




 18%|█▊        | 14934/85219 [00:20<01:18, 892.05it/s] 




 18%|█▊        | 15083/85219 [00:20<01:25, 816.65it/s]




 18%|█▊        | 15208/85219 [00:21<01:37, 720.54it/s]




 18%|█▊        | 15313/85219 [00:21<01:46, 657.28it/s]




 18%|█▊        | 15403/85219 [00:21<01:42, 677.95it/s]




 18%|█▊        | 15488/85219 [00:21<01:52, 620.86it

 27%|██▋       | 22599/85219 [00:33<02:48, 372.22it/s]




 27%|██▋       | 22661/85219 [00:34<02:32, 411.27it/s]




 27%|██▋       | 22711/85219 [00:34<02:36, 398.63it/s]




 27%|██▋       | 22762/85219 [00:34<02:41, 386.91it/s]




 27%|██▋       | 22857/85219 [00:34<02:13, 466.02it/s]




 27%|██▋       | 22961/85219 [00:34<01:52, 552.79it/s]




 27%|██▋       | 23036/85219 [00:34<01:45, 587.51it/s]




 27%|██▋       | 23105/85219 [00:34<01:46, 582.43it/s]




 27%|██▋       | 23196/85219 [00:34<01:35, 651.54it/s]




 27%|██▋       | 23269/85219 [00:35<01:44, 594.47it/s]




 27%|██▋       | 23344/85219 [00:35<01:38, 629.41it/s]




 27%|██▋       | 23412/85219 [00:35<01:45, 585.16it/s]




 28%|██▊       | 23475/85219 [00:35<01:57, 524.83it/s]




 28%|██▊       | 23532/85219 [00:35<02:16, 452.96it/s]




 28%|██▊       | 23600/85219 [00:35<02:06, 487.64it/s]




 28%|██▊       | 23685/85219 [00:35<01:50, 555.69it/s]




 28%|██▊       | 23781/85219 [00:35<01:36, 633.72it/s]



 35%|███▌      | 29974/85219 [00:48<02:00, 458.42it/s]




 35%|███▌      | 30065/85219 [00:48<01:42, 536.09it/s]




 35%|███▌      | 30152/85219 [00:48<01:35, 575.06it/s]




 36%|███▌      | 30261/85219 [00:49<01:22, 665.11it/s]




 36%|███▌      | 30340/85219 [00:49<01:19, 689.95it/s]




 36%|███▌      | 30418/85219 [00:49<01:19, 685.75it/s]




 36%|███▌      | 30494/85219 [00:49<01:17, 703.61it/s]




 36%|███▌      | 30569/85219 [00:49<01:28, 615.04it/s]




 36%|███▌      | 30636/85219 [00:49<01:36, 565.39it/s]




 36%|███▌      | 30697/85219 [00:49<01:40, 540.30it/s]




 36%|███▌      | 30755/85219 [00:49<01:40, 541.66it/s]




 36%|███▌      | 30822/85219 [00:49<01:37, 557.86it/s]




 36%|███▌      | 30880/85219 [00:50<01:37, 555.80it/s]




 36%|███▋      | 30937/85219 [00:50<01:39, 545.20it/s]




 36%|███▋      | 30993/85219 [00:50<01:52, 482.54it/s]




 36%|███▋      | 31061/85219 [00:50<01:42, 526.21it/s]




 37%|███▋      | 31117/85219 [00:50<02:07, 423.71it/s]



 44%|████▍     | 37394/85219 [01:02<01:07, 706.00it/s]




 44%|████▍     | 37487/85219 [01:03<01:02, 758.99it/s]




 44%|████▍     | 37589/85219 [01:03<00:58, 818.08it/s]




 44%|████▍     | 37677/85219 [01:03<01:19, 601.64it/s]




 44%|████▍     | 37750/85219 [01:03<01:23, 570.55it/s]




 44%|████▍     | 37855/85219 [01:03<01:16, 617.39it/s]




 45%|████▍     | 37944/85219 [01:03<01:09, 676.64it/s]




 45%|████▍     | 38019/85219 [01:03<01:14, 633.54it/s]




 45%|████▍     | 38088/85219 [01:04<01:24, 555.57it/s]




 45%|████▍     | 38173/85219 [01:04<01:16, 613.31it/s]




 45%|████▍     | 38240/85219 [01:04<01:39, 473.90it/s]




 45%|████▍     | 38335/85219 [01:04<01:24, 554.37it/s]




 45%|████▌     | 38402/85219 [01:04<01:20, 580.94it/s]




 45%|████▌     | 38469/85219 [01:04<01:30, 519.30it/s]




 45%|████▌     | 38532/85219 [01:04<01:25, 545.98it/s]




 45%|████▌     | 38664/85219 [01:04<01:10, 659.53it/s]




 45%|████▌     | 38745/85219 [01:05<01:30, 511.30it/s]



...

 61%|██████▏   | 52254/85219 [01:33<01:02, 528.24it/s]




 61%|██████▏   | 52312/85219 [01:33<01:01, 536.30it/s]




 61%|██████▏   | 52381/85219 [01:33<00:57, 574.67it/s]




 62%|██████▏   | 52442/85219 [01:33<00:59, 555.49it/s]




 62%|██████▏   | 52500/85219 [01:33<00:58, 555.87it/s]




 62%|██████▏   | 52558/85219 [01:33<01:03, 512.40it/s]




 62%|██████▏   | 52626/85219 [01:33<01:02, 521.23it/s]




 62%|██████▏   | 52773/85219 [01:33<00:50, 636.93it/s]




 62%|██████▏   | 52851/85219 [01:34<00:52, 616.96it/s]




 62%|██████▏   | 52923/85219 [01:34<00:58, 550.34it/s]




 62%|██████▏   | 52987/85219 [01:34<00:57, 563.05it/s]




 62%|██████▏   | 53049/85219 [01:34<01:00, 531.92it/s]




 62%|██████▏   | 53107/85219 [01:34<01:29, 357.70it/s]




 62%|██████▏   | 53154/85219 [01:34<01:51, 288.66it/s]




 62%|██████▏   | 53193/85219 [01:35<02:05, 254.42it/s]




 62%|██████▏   | 53226/85219 [01:35<02:01, 263.15it/s]




 62%|██████▏   | 53258/85219 [01:35<02:13, 239.99it/s]



 69%|██████▉   | 59197/85219 [01:47<00:42, 605.58it/s]




 70%|██████▉   | 59263/85219 [01:47<00:45, 568.47it/s]




 70%|██████▉   | 59362/85219 [01:48<00:40, 646.31it/s]




 70%|██████▉   | 59444/85219 [01:48<00:37, 685.22it/s]




 70%|██████▉   | 59518/85219 [01:48<00:36, 696.99it/s]




 70%|██████▉   | 59592/85219 [01:48<00:39, 645.33it/s]




 70%|███████   | 59685/85219 [01:48<00:38, 663.14it/s]




 70%|███████   | 59760/85219 [01:48<00:37, 685.09it/s]




 70%|███████   | 59844/85219 [01:48<00:35, 715.09it/s]




 70%|███████   | 59931/85219 [01:48<00:33, 754.75it/s]




 70%|███████   | 60021/85219 [01:48<00:34, 739.26it/s]




 71%|███████   | 60097/85219 [01:49<00:57, 437.75it/s]




 71%|███████   | 60157/85219 [01:49<00:58, 431.27it/s]




 71%|███████   | 60212/85219 [01:49<01:03, 394.03it/s]




 71%|███████   | 60260/85219 [01:49<01:06, 377.99it/s]




 71%|███████   | 60304/85219 [01:49<01:08, 363.54it/s]




 71%|███████   | 60345/85219 [01:50<01:23, 297.48it/s]



 78%|███████▊  | 66205/85219 [02:02<00:48, 390.89it/s]




 78%|███████▊  | 66287/85219 [02:02<00:42, 450.52it/s]




 78%|███████▊  | 66338/85219 [02:02<00:59, 315.83it/s]




 78%|███████▊  | 66379/85219 [02:03<00:56, 334.45it/s]




 78%|███████▊  | 66425/85219 [02:03<00:51, 363.55it/s]




 78%|███████▊  | 66470/85219 [02:03<00:50, 373.14it/s]




 78%|███████▊  | 66512/85219 [02:03<00:51, 362.41it/s]




 78%|███████▊  | 66552/85219 [02:03<01:08, 274.20it/s]




 78%|███████▊  | 66585/85219 [02:03<01:14, 250.22it/s]




 78%|███████▊  | 66615/85219 [02:03<01:16, 244.01it/s]




 78%|███████▊  | 66643/85219 [02:04<01:24, 220.91it/s]




 78%|███████▊  | 66668/85219 [02:04<01:23, 221.09it/s]




 78%|███████▊  | 66692/85219 [02:04<01:50, 168.00it/s]




 78%|███████▊  | 66715/85219 [02:04<01:43, 178.48it/s]




 78%|███████▊  | 66736/85219 [02:04<02:07, 145.07it/s]




 78%|███████▊  | 66772/85219 [02:04<01:50, 166.50it/s]




 78%|███████▊  | 66825/85219 [02:04<01:28, 207.66it/s]



 85%|████████▍ | 72112/85219 [02:18<00:35, 369.72it/s]




 85%|████████▍ | 72165/85219 [02:18<00:33, 388.52it/s]




 85%|████████▍ | 72222/85219 [02:18<00:30, 429.27it/s]




 85%|████████▍ | 72274/85219 [02:18<00:30, 427.65it/s]




 85%|████████▍ | 72324/85219 [02:18<00:32, 391.90it/s]




 85%|████████▍ | 72369/85219 [02:18<00:36, 356.53it/s]




 85%|████████▌ | 72439/85219 [02:18<00:32, 388.59it/s]




 85%|████████▌ | 72482/85219 [02:18<00:33, 380.34it/s]




 85%|████████▌ | 72523/85219 [02:19<00:35, 356.80it/s]




 85%|████████▌ | 72599/85219 [02:19<00:30, 416.95it/s]




 85%|████████▌ | 72658/85219 [02:19<00:27, 455.51it/s]




 85%|████████▌ | 72727/85219 [02:19<00:26, 475.40it/s]




 85%|████████▌ | 72784/85219 [02:19<00:25, 487.27it/s]




 85%|████████▌ | 72855/85219 [02:19<00:23, 521.43it/s]




 86%|████████▌ | 72910/85219 [02:19<00:23, 528.15it/s]




 86%|████████▌ | 72965/85219 [02:19<00:27, 447.40it/s]




 86%|████████▌ | 73014/85219 [02:20<00:30, 397.86it/s]



 94%|█████████▍| 80285/85219 [02:32<00:07, 702.22it/s]




 94%|█████████▍| 80387/85219 [02:32<00:06, 767.39it/s]




 94%|█████████▍| 80470/85219 [02:32<00:06, 710.22it/s]




 95%|█████████▍| 80576/85219 [02:32<00:05, 786.77it/s]




 95%|█████████▍| 80661/85219 [02:32<00:06, 738.72it/s]




 95%|█████████▍| 80740/85219 [02:32<00:06, 716.87it/s]




 95%|█████████▍| 80816/85219 [02:33<00:08, 510.10it/s]




 95%|█████████▍| 80909/85219 [02:33<00:07, 579.19it/s]




 95%|█████████▌| 80991/85219 [02:33<00:06, 635.08it/s]




 95%|█████████▌| 81064/85219 [02:33<00:08, 515.46it/s]




 95%|█████████▌| 81126/85219 [02:33<00:08, 483.14it/s]




 95%|█████████▌| 81196/85219 [02:33<00:07, 521.71it/s]




 95%|█████████▌| 81318/85219 [02:34<00:06, 629.61it/s]




 96%|█████████▌| 81396/85219 [02:34<00:07, 491.57it/s]




 96%|█████████▌| 81470/85219 [02:34<00:07, 516.74it/s]




 96%|█████████▌| 81549/85219 [02:34<00:06, 573.11it/s]




 96%|█████████▌| 81616/85219 [02:34<00:06, 534.61it/s]



In [230]:
pickle.dump(M_doc2vec_train, open('/mnt/dump/dump_pipeline/M_doc2Vec_train.pickle', 'wb'))
pickle.dump(M_doc2vec_test,  open('/mnt/dump/dump_pipeline/M_doc2vec_test.pickle', 'wb'))

In [229]:
M_doc2vec_test = M_doc2Vec_test

In [214]:
# simple_preprocess('sdf lskjdf lksdjf')

[1779]

[641]

In [179]:
vect_doc2vec.infer_vector(words)

array([-0.13571584,  0.3499744 , -0.04202783,  0.21361765, -0.1456287 ,
        0.35336843, -0.03166652, -0.15306666,  0.11268553,  0.17767656,
       -0.39363706,  0.087484  , -0.03432494, -0.4482898 , -0.18006206,
        0.5096276 ,  0.07015085, -0.05675733, -0.14410444, -0.0205825 ,
        0.08255273,  0.24701715, -0.21387291,  0.34846246,  0.06285395,
        0.37631196,  0.0649631 ,  0.2685452 , -0.04495054, -0.02767053,
       -0.0965929 , -0.08395821,  0.32579374,  0.08346151,  0.11441112,
       -0.15678144, -0.00495434, -0.04642475,  0.2616632 , -0.16768491,
       -0.07114959,  0.10572839, -0.32332993, -0.11214775, -0.17758787,
        0.08280778,  0.07631755, -0.05473506,  0.14580464,  0.13253057,
        0.05979443,  0.06537861,  0.06792891, -0.18005669,  0.06069762,
        0.07514177, -0.21833692,  0.27156472,  0.1358251 ,  0.15479502,
       -0.02934217, -0.03083587,  0.05743814, -0.38069576, -0.19944763,
        0.15103269,  0.26865512,  0.00414145,  0.14888793,  0.13

In [165]:
vect_doc2vec.infer_vector()

In [338]:
y_train, X_train = vec_for_learning(vect_doc2vec, train_tagged)
y_test, X_test = vec_for_learning(vect_doc2vec, test_tagged)

ValueError: too many values to unpack (expected 2)

In [ ]:
pickle.dump(M_doc2vec_vect_train, open('/mnt/dump/dump_pipeline/M_doc2vec_vect_train.pickle', 'wb'))
pickle.dump(M_doc2vec_vect_test,  open('/mnt/dump/dump_pipeline/M_doc2vec_vect_test.pickle', 'wb'))

LDA

In [ ]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=50, random_state=0)
M_lda_count_vect_train = lda_tf.fit_transform(M_count_vect_train)
M_lda_count_vect_test = lda_tf.transform(M_count_vect_test)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=50, random_state=0)
M_lda_tfidf_vect_train = lda_tfidf.fit_transform(M_tf_idf_vect_train)
M_lda_tfidf_vect_test = lda_tfidf.transform(M_tf_idf_vect_test)

In [241]:
M_lda_count_vect_train 

NameError: name 'M_lda_count_vect_train' is not defined

In [ ]:
pickle.dump(M_lda_tfidf_vect_train, open('/mnt/dump/dump_pipeline/M_lda_vect_train.pickle', 'wb'))
pickle.dump(M_lda_tfidf_vect_test,  open('/mnt/dump/dump_pipeline/M_lda_vect_test.pickle', 'wb'))

Models

In [17]:
%%time
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

# Модели
model_LR = LogisticRegression(n_jobs = N_JOBS, solver='warn')
model_XGB = XGBClassifier(n_jobs = N_JOBS)
model_RF= RandomForestClassifier(n_jobs = N_JOBS)

CPU times: user 23.5 ms, sys: 0 ns, total: 23.5 ms
Wall time: 22.4 ms


Описание

    в М_название_векторайзера_train/test - лежат ембединги
    в df_train['title'] и df_test['title'] лежит y
    Учим на М, предсказываем y - все

In [12]:
with open('/mnt/dump/dump_pipeline/M_word2vec_google_vect_train.pickle', 'rb') as f:
    M_word2vec_google_vect_train = pickle.load(f)

with open('/mnt/dump/dump_pipeline/M_word2vec_google_vect_test.pickle', 'rb') as f:
    M_word2vec_google_vect_test = pickle.load(f)

In [13]:
with open('/mnt/dump/dump_pipeline/M_word2vec_our_vect_train.pickle', 'rb') as f:
    M_word2vec_our_vect_train = pickle.load(f)

with open('/mnt/dump/dump_pipeline/M_word2vec_our_vect_test.pickle', 'rb') as f:
    M_word2vec_our_vect_test = pickle.load(f)

In [14]:
with open('/mnt/dump/dump_pipeline/M_count_vect_train.pickle', 'rb') as f:
    M_count_vect_train = pickle.load(f)

with open('/mnt/dump/dump_pipeline/M_count_vect_test.pickle', 'rb') as f:
    M_count_vect_test = pickle.load(f)

In [15]:
with open('/mnt/dump/dump_pipeline/M_tf_idf_vect_train.pickle', 'rb') as f:
    M_tf_idf_vect_train = pickle.load(f)

with open('/mnt/dump/dump_pipeline/M_tf_idf_vect_test.pickle', 'rb') as f:
    M_tf_idf_vect_test = pickle.load(f)

In [18]:
embedings_dict = {
    'train' : {'count-vectorizer': M_count_vect_train,
               'tf-idf vectorizer': M_tf_idf_vect_train,
               'word2vec (fitted on our data)': M_word2vec_our_vect_train,
               'word2vec (fitted on google news)': M_word2vec_google_vect_train,
               #'doc2vec': M_doc2vec_vect_train
              },
              
    'test'  : {'count-vectorizer': M_count_vect_test,
               'tf-idf vectorizer': M_tf_idf_vect_test,
               'word2vec (fitted on our data)': M_word2vec_our_vect_test,
               'word2vec (fitted on google news)': M_word2vec_google_vect_test,
               #'doc2vec': M_doc2vec_vect_test
              }
    }


model_dict = {'Logistic regression': model_LR,
              'XGBoost': model_XGB,
              'Random Forest': model_RF
             }

fff

In [ ]:
%%time
y_train = df_train['title'].get_values()
y_test = df_test['title'].get_values()

for embeding_name in embedings_dict['train'].keys():
    M_train = embedings_dict['train'][embeding_name]
    M_test  = embedings_dict['test'][embeding_name]
    
    
    for model_name, model in model_dict.items():
        print(embeding_name, model_name)
        model.fit(M_train, y_train)
        
        p_test  = model.predict_proba(M_test)
        p_train = model.predict_proba(M_train)
        
        res = {'p_train': p_train,
               'p_test':  p_test,
               }
        
        pickle.dump(res, open('/mnt/dump/dump_pipeline/result_'+embeding_name+"|"+model_name+'.pickle', 'wb'))

tf-idf vectorizer XGBoost


In [22]:
M_train.shape, y_train.shape

((85219, 300), (85219,))

In [32]:
M_count_vect_test[:100].shape

(100, 300)

#### count vect + RF

In [56]:
model = RandomForestClassifier(n_jobs=40, verbose=1)#, solver='sag')
model.fit(M_count_vect_train, df_train['title'].get_values())

/home/autodeploy/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   8 out of  10 | elapsed:  1.9min remaining:   28.2s
[Parallel(n_jobs=40)]: Done  10 out of  10 | elapsed:  2.3min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=40,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [60]:
# model.classes_

In [43]:
res_test = model.predict_proba(M_count_vect_test)
res_train = model.predict_proba(M_count_vect_train)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.4s remaining:    1.6s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.8s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    2.6s remaining:   10.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    5.5s finished


In [59]:
M_word2vec_our_vect_train

(10229, 300)

In [ ]:
dump_res(res_train, res_test, 'count_vect_rf')

TF-idf + RF

In [57]:
model = RandomForestClassifier(n_jobs=40, verbose=1)#, solver='sag')
model.fit(M_tf_idf_vect_train, df_train['title'].get_values())

res_test = model.predict_proba(M_tf_idf_vect_train)
res_train = model.predict_proba(M_tf_idf_vect_test)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    2.0s remaining:    8.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    4.5s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.2s remaining:    1.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.6s finished


##### Word2Vec (our)

In [61]:
%%time
model = RandomForestClassifier(n_jobs=40, verbose=1)#, solver='sag')
model.fit(M_word2vec_our_vect_train, df_train['title'].get_values())

res_test = model.predict_proba(M_word2vec_our_vect_train)
res_train = model.predict_proba(M_word2vec_our_vect_test)
dump_res(res_train, res_test, 'word2vect_our')

/home/autodeploy/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   8 out of  10 | elapsed:   28.9s remaining:    7.2s
[Parallel(n_jobs=40)]: Done  10 out of  10 | elapsed:   29.9s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    1.9s remaining:    7.5s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    4.7s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.2s remaining:    0.9s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.6s finished


CPU times: user 4min 57s, sys: 0 ns, total: 4min 57s
Wall time: 38.1 s


##### Word2Vec (pretrained google news)

In [62]:
%%time
model = RandomForestClassifier(n_jobs=40, verbose=1)#, solver='sag')
model.fit(M_word2vec_google_vect_train, df_train['title'].get_values())

res_test = model.predict_proba(M_word2vec_google_vect_train)
res_train = model.predict_proba(M_word2vec_google_vect_test)
dump_res(res_train, res_test, 'word2vect_google_news')

/home/autodeploy/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   8 out of  10 | elapsed:   29.5s remaining:    7.4s
[Parallel(n_jobs=40)]: Done  10 out of  10 | elapsed:   30.3s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    1.9s remaining:    7.5s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    5.1s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.2s remaining:    1.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.6s finished


CPU times: user 5min 3s, sys: 0 ns, total: 5min 3s
Wall time: 39.2 s


In [54]:
df_test['label']

44           53
45           53
46           53
47           53
52           56
53           56
54           56
55           56
61           58
62           58
63           58
256         105
257         105
258         105
259         105
278         110
279         110
280         110
281         110
355         128
356         128
357         128
358         128
359         128
413         140
414         140
415         140
416         140
483         157
484         157
          ...  
102659    19918
102660    19918
102664    19920
102665    19920
102671    19922
102672    19922
102673    19922
102676    19924
102677    19924
102678    19924
102679    19924
102680    19924
102681    19924
102682    19924
102683    19924
102684    19925
102685    19925
102686    19925
102687    19925
102688    19925
102689    19925
102729    19938
102730    19938
102731    19938
102735    19940
102736    19940
102737    19940
102738    19940
102739    19940
102740    19940
Name: label, Length: 102

In [52]:
np.mean(list(sorted(model.classes_)) == model.classes_)

1.0

In [45]:
def dump_res(res_train, res_test, name_pref=None):
    path = '/mnt/dump/final/'
    pickle.dump(res_train, open(path+name_pref+'_train.pickle', 'wb'))
    pickle.dump(res_test, open(path+name_pref+'_test.pickle', 'wb'))
    print('done')
dump_res(res_train, res_test, 'count_vect_rf')

In [44]:
pickle.dump('count_vect_train,res_train

(10229, 2007)

In [64]:
# 1

In [28]:
10  - 2
100 - 50
?1000 - 50*25
?10000 - 50*25*25
?100000 - 50*25*25*25

AttributeError: 'LabelEncoder' object has no attribute 'fit_tra'

In [105]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(256, input_shape=(300,)),
    Activation('relu'),
    Dropout(0.1)
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [140]:
def prepare_data(M_, df):
    M = M_.copy()
#     if hasattr(M, 'toarray'):
#         M = M.toarray()
    classes = list(df['title'].unique())
    
    X, y = [], []
    label_dict = {v: k for k, v in enumerate(df['label'].get_values())}
    for c, (label, obj) in enumerate(df.groupby('label')):
        ind = label_dict[label]
        X.append(M[ind, :])
        temp_y = np.zeros(len(classes))
        for i in obj['title'].get_values():
            temp_y[i] = 1
        y.append(temp_y)
    return np.array(X), np.array(y)



In [107]:
X_train, y_train = prepare_data(M_count_vect_train, df_train)
X_test, y_test = prepare_data(M_count_vect_test, df_test)

model.fit(x = X_train, y = y_train, epochs=10)

Epoch 1/10
17377/17377 [==============================] - 5s 265us/step - loss: 29.7125 - acc: 0.0754
Epoch 2/10
17377/17377 [==============================] - 5s 308us/step - loss: 25.4864 - acc: 0.1124
Epoch 3/10
17377/17377 [==============================] - 5s 276us/step - loss: 24.2719 - acc: 0.1194
Epoch 4/10
17377/17377 [==============================] - 5s 276us/step - loss: 23.6343 - acc: 0.1211
Epoch 5/10
17377/17377 [==============================] - 5s 273us/step - loss: 23.2796 - acc: 0.1240
Epoch 6/10
17377/17377 [==============================] - 5s 284us/step - loss: 23.0557 - acc: 0.1222
Epoch 7/10
17377/17377 [==============================] - 5s 285us/step - loss: 22.8795 - acc: 0.1232
Epoch 8/10
17377/17377 [==============================] - 5s 272us/step - loss: 22.7763 - acc: 0.1253
Epoch 9/10
17377/17377 [==============================] - 5s 296us/step - loss: 22.7083 - acc: 0.1249
Epoch 10/10
17377/17377 [==============================] - 4s 256us/step - loss: 2

In [110]:
%%time
p_train = model.predict_proba(X_train)
p_test = model.predict_proba(X_test)

In [141]:
%%time
from sklearn.metrics import roc_auc_score, average_precision_score
def get_roc_and_ap(y, p):
    roc_list, ap_list = [],[]
    for p_,y_ in zip(p, y):
        roc_list.append(roc_auc_score(y_, p_))
        ap_list.append(average_precision_score(y_, p_))
    return roc_list, ap_list

# print('test')
# roc_list, ap_list = get_roc_and_ap(y_test, p_test)
# print('ROC: ', np.mean(roc_list))
# print('AP: ', np.mean(ap_list))

# print('train')
# roc_list, ap_list = get_roc_and_ap(y_train, p_train)
# print('ROC: ', np.mean(roc_list))
# print('AP: ', np.mean(ap_list))


CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 42.4 µs


In [242]:
1

1

In [142]:
def calculate_all(M_train, M_test, embeding_name = "none", model = None):
    print(embeding_name)
    X_train, y_train = prepare_data(M_train, df_train)
    X_test, y_test = prepare_data(M_test, df_test)

    model.fit(x = X_train, y = y_train, epochs=10)

    p_train = model.predict_proba(X_train)
    p_test = model.predict_proba(X_test)
    
    print('test')
    roc_list, ap_list = get_roc_and_ap(y_test, p_test)
    print('ROC: ', np.mean(roc_list))
    print('AP: ', np.mean(ap_list))

    print('train')
    roc_list, ap_list = get_roc_and_ap(y_train, p_train)
    print('ROC: ', np.mean(roc_list))
    print('AP: ', np.mean(ap_list))

In [126]:
%%time

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(100, input_shape=(300,)),
    Activation('relu'),
#     Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

calculate_all(M_count_vect_train.toarray(), M_count_vect_test.toarray(), 'Count-vect + NN', model = model)

Count-vect + NN
Epoch 1/10
17377/17377 [==============================] - 6s 324us/step - loss: 22.6358 - acc: 0.1238
Epoch 2/10
17377/17377 [==============================] - 6s 320us/step - loss: 22.6158 - acc: 0.1231
Epoch 3/10
17377/17377 [==============================] - 5s 312us/step - loss: 22.6395 - acc: 0.1229
Epoch 4/10
17377/17377 [==============================] - 6s 320us/step - loss: 22.6569 - acc: 0.1192
Epoch 5/10
17377/17377 [==============================] - 5s 302us/step - loss: 22.6762 - acc: 0.1216
Epoch 6/10
17377/17377 [==============================] - 6s 320us/step - loss: 22.7189 - acc: 0.1183
Epoch 7/10
17377/17377 [==============================] - 5s 298us/step - loss: 22.7300 - acc: 0.1211
Epoch 8/10
17377/17377 [==============================] - 5s 289us/step - loss: 22.7948 - acc: 0.1185
Epoch 9/10
17377/17377 [==============================] - 5s 289us/step - loss: 22.8509 - acc: 0.1190
Epoch 10/10
17377/17377 [==============================] - 5s 288u

In [146]:
# # M_tf_idf_vect_train
# _, __ = prepare_data(M_tf_idf_vect_train.toarray(), df_train)
# __[10,:].sum()

5.0

In [147]:
%%time

model = Sequential([
    Dense(100, input_shape=(300,)),
    Activation('relu'),
#     Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

calculate_all(M_tf_idf_vect_train.toarray(), M_tf_idf_vect_test.toarray(), 'TF-IDF + NN', model = model)

TF-IDF + NN
Epoch 1/10
17377/17377 [==============================] - 6s 324us/step - loss: 31.9047 - acc: 0.0390
Epoch 2/10
17377/17377 [==============================] - 5s 298us/step - loss: 27.9659 - acc: 0.0836
Epoch 3/10
17377/17377 [==============================] - 5s 289us/step - loss: 26.0176 - acc: 0.1005
Epoch 4/10
17377/17377 [==============================] - 5s 293us/step - loss: 24.7759 - acc: 0.1146
Epoch 5/10
17377/17377 [==============================] - 5s 284us/step - loss: 23.8661 - acc: 0.1243
Epoch 6/10
17377/17377 [==============================] - 5s 290us/step - loss: 23.1847 - acc: 0.1300
Epoch 7/10
17377/17377 [==============================] - 5s 284us/step - loss: 22.6233 - acc: 0.1372
Epoch 8/10
17377/17377 [==============================] - 5s 277us/step - loss: 22.1692 - acc: 0.1414
Epoch 9/10
17377/17377 [==============================] - 5s 279us/step - loss: 21.7793 - acc: 0.1444
Epoch 10/10
17377/17377 [==============================] - 5s 273us/st

In [150]:
%%time

model = Sequential([
    Dense(100, input_shape=(300,)),
    Activation('relu'),
#     Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


calculate_all(M_word2vec_our_vect_train,
              M_word2vec_our_vect_test,
              'Word2Vec(our) + NN',
              model = model)

Word2Vec(our) + NN
Epoch 1/10
17377/17377 [==============================] - 5s 297us/step - loss: 26.3806 - acc: 0.0940
Epoch 2/10
17377/17377 [==============================] - 5s 274us/step - loss: 22.3444 - acc: 0.1307
Epoch 3/10
17377/17377 [==============================] - 5s 269us/step - loss: 21.1301 - acc: 0.1457
Epoch 4/10
17377/17377 [==============================] - 5s 284us/step - loss: 20.4581 - acc: 0.1501
Epoch 5/10
17377/17377 [==============================] - 5s 270us/step - loss: 20.0168 - acc: 0.1532
Epoch 6/10
17377/17377 [==============================] - 5s 272us/step - loss: 19.7066 - acc: 0.1539
Epoch 7/10
17377/17377 [==============================] - 5s 271us/step - loss: 19.4709 - acc: 0.1580
Epoch 8/10
17377/17377 [==============================] - 5s 277us/step - loss: 19.2987 - acc: 0.1548
Epoch 9/10
17377/17377 [==============================] - 5s 272us/step - loss: 19.1386 - acc: 0.1553
Epoch 10/10
17377/17377 [==============================] - 5s 2

In [149]:
%%time

model = Sequential([
    Dense(100, input_shape=(300,)),
    Activation('relu'),
#     Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


calculate_all(M_word2vec_google_vect_train,
              M_word2vec_google_vect_test,
              'Word2Vec(google news) + NN',
              model=model)

Word2Vec(google news) + NN
Epoch 1/10
17377/17377 [==============================] - 5s 316us/step - loss: 31.5423 - acc: 0.0392
Epoch 2/10
17377/17377 [==============================] - 5s 288us/step - loss: 27.7391 - acc: 0.0807
Epoch 3/10
17377/17377 [==============================] - 5s 279us/step - loss: 25.7104 - acc: 0.1044
Epoch 4/10
17377/17377 [==============================] - 4s 251us/step - loss: 24.4602 - acc: 0.1175
Epoch 5/10
17377/17377 [==============================] - 5s 264us/step - loss: 23.6082 - acc: 0.1254
Epoch 6/10
17377/17377 [==============================] - 5s 266us/step - loss: 22.9617 - acc: 0.1365
Epoch 7/10
17377/17377 [==============================] - 5s 267us/step - loss: 22.4478 - acc: 0.1412
Epoch 8/10
17377/17377 [==============================] - 5s 287us/step - loss: 22.0305 - acc: 0.1430
Epoch 9/10
17377/17377 [==============================] - 5s 261us/step - loss: 21.6741 - acc: 0.1440
Epoch 10/10
17377/17377 [==============================

Another NN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(256, input_shape=(300,)),
    Activation('relu'),
    Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [152]:
calculate_all(M_count_vect_train.toarray(), M_count_vect_test.toarray(), 'Count-vect + NN', model = model)

Count-vect + NN
Epoch 1/10
17377/17377 [==============================] - 9s 522us/step - loss: 29.4357 - acc: 0.0826
Epoch 2/10
17377/17377 [==============================] - 8s 450us/step - loss: 25.1246 - acc: 0.1223
Epoch 3/10
17377/17377 [==============================] - 8s 474us/step - loss: 23.9309 - acc: 0.1311
Epoch 4/10
17377/17377 [==============================] - 8s 463us/step - loss: 23.2157 - acc: 0.1352
Epoch 5/10
17377/17377 [==============================] - 8s 469us/step - loss: 22.8626 - acc: 0.1332
Epoch 6/10
17377/17377 [==============================] - 8s 483us/step - loss: 22.5489 - acc: 0.1326
Epoch 7/10
17377/17377 [==============================] - 8s 485us/step - loss: 22.3661 - acc: 0.1373
Epoch 8/10
17377/17377 [==============================] - 8s 469us/step - loss: 22.2788 - acc: 0.1359
Epoch 9/10
17377/17377 [==============================] - 8s 445us/step - loss: 22.1515 - acc: 0.1370
Epoch 10/10
17377/17377 [==============================] - 8s 467u

In [153]:
calculate_all(M_tf_idf_vect_train.toarray(), M_tf_idf_vect_test.toarray(), 'TF-IDF + NN', model = model)

TF-IDF + NN
Epoch 1/10
17377/17377 [==============================] - 9s 514us/step - loss: 30.5665 - acc: 0.0559
Epoch 2/10
17377/17377 [==============================] - 8s 441us/step - loss: 26.3012 - acc: 0.1054
Epoch 3/10
17377/17377 [==============================] - 8s 462us/step - loss: 24.3982 - acc: 0.1217
Epoch 4/10
17377/17377 [==============================] - 8s 456us/step - loss: 23.2259 - acc: 0.1330
Epoch 5/10
17377/17377 [==============================] - 8s 458us/step - loss: 22.3827 - acc: 0.1398
Epoch 6/10
17377/17377 [==============================] - 8s 483us/step - loss: 21.7421 - acc: 0.1484
Epoch 7/10
17377/17377 [==============================] - 8s 473us/step - loss: 21.2243 - acc: 0.1542
Epoch 8/10
17377/17377 [==============================] - 8s 476us/step - loss: 20.7846 - acc: 0.1595
Epoch 9/10
17377/17377 [==============================] - 8s 476us/step - loss: 20.4110 - acc: 0.1584
Epoch 10/10
17377/17377 [==============================] - 8s 475us/st

In [154]:
calculate_all(M_word2vec_our_vect_train,
              M_word2vec_our_vect_test,
              'Word2Vec(our) + NN',
              model = model)

Word2Vec(our) + NN
Epoch 1/10
17377/17377 [==============================] - 12s 675us/step - loss: 25.4662 - acc: 0.1147
Epoch 2/10
17377/17377 [==============================] - 11s 648us/step - loss: 21.7189 - acc: 0.1450
Epoch 3/10
17377/17377 [==============================] - 11s 658us/step - loss: 20.6715 - acc: 0.1538
Epoch 4/10
17377/17377 [==============================] - 12s 668us/step - loss: 20.1101 - acc: 0.1563
Epoch 5/10
17377/17377 [==============================] - 12s 663us/step - loss: 19.7423 - acc: 0.1582
Epoch 6/10
17377/17377 [==============================] - 11s 658us/step - loss: 19.4473 - acc: 0.1582
Epoch 7/10
17377/17377 [==============================] - 11s 637us/step - loss: 19.2419 - acc: 0.1573
Epoch 8/10
17377/17377 [==============================] - 11s 636us/step - loss: 19.0979 - acc: 0.1599
Epoch 9/10
17377/17377 [==============================] - 11s 632us/step - loss: 18.9559 - acc: 0.1579
Epoch 10/10
17377/17377 [=============================

In [155]:
calculate_all(M_word2vec_google_vect_train,
              M_word2vec_google_vect_test,
              'Word2Vec(google news) + NN',
              model=model)

Word2Vec(google news) + NN
Epoch 1/10
17377/17377 [==============================] - 12s 692us/step - loss: 30.5954 - acc: 0.0546
Epoch 2/10
17377/17377 [==============================] - 12s 674us/step - loss: 26.3109 - acc: 0.1043
Epoch 3/10
17377/17377 [==============================] - 11s 661us/step - loss: 24.3905 - acc: 0.1208
Epoch 4/10
17377/17377 [==============================] - 12s 679us/step - loss: 23.2563 - acc: 0.1315
Epoch 5/10
17377/17377 [==============================] - 11s 660us/step - loss: 22.4886 - acc: 0.1392
Epoch 6/10
17377/17377 [==============================] - 12s 664us/step - loss: 21.8988 - acc: 0.1435
Epoch 7/10
17377/17377 [==============================] - 12s 667us/step - loss: 21.4334 - acc: 0.1476
Epoch 8/10
17377/17377 [==============================] - 12s 685us/step - loss: 21.0882 - acc: 0.1532
Epoch 9/10
17377/17377 [==============================] - 11s 642us/step - loss: 20.7778 - acc: 0.1563
Epoch 10/10
17377/17377 [=====================

Another NN

In [156]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(512, input_shape=(300,)),
    Activation('relu'),
    Dropout(0.1),
    Dense(256),
    Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [157]:
calculate_all(M_count_vect_train.toarray(), M_count_vect_test.toarray(), 'Count-vect + NN', model = model)

Count-vect + NN
Epoch 1/10
17377/17377 [==============================] - 7s 419us/step - loss: 29.4557 - acc: 0.0923
Epoch 2/10
17377/17377 [==============================] - 7s 393us/step - loss: 25.0875 - acc: 0.1238
Epoch 3/10
17377/17377 [==============================] - 7s 388us/step - loss: 23.7112 - acc: 0.1376
Epoch 4/10
17377/17377 [==============================] - 7s 383us/step - loss: 22.8761 - acc: 0.1448
Epoch 5/10
17377/17377 [==============================] - 7s 376us/step - loss: 22.2130 - acc: 0.1470
Epoch 6/10
17377/17377 [==============================] - 7s 381us/step - loss: 21.7145 - acc: 0.1504
Epoch 7/10
17377/17377 [==============================] - 6s 372us/step - loss: 21.3470 - acc: 0.1579
Epoch 8/10
17377/17377 [==============================] - 7s 392us/step - loss: 21.0587 - acc: 0.1589
Epoch 9/10
17377/17377 [==============================] - 7s 383us/step - loss: 20.8295 - acc: 0.1629
Epoch 10/10
17377/17377 [==============================] - 7s 385u

In [158]:
calculate_all(M_tf_idf_vect_train.toarray(), M_tf_idf_vect_test.toarray(), 'TF-IDF + NN', model = model)

TF-IDF + NN
Epoch 1/10
17377/17377 [==============================] - 6s 369us/step - loss: 25.7895 - acc: 0.1104
Epoch 2/10
17377/17377 [==============================] - 7s 380us/step - loss: 23.0389 - acc: 0.1377
Epoch 3/10
17377/17377 [==============================] - 7s 378us/step - loss: 22.3223 - acc: 0.1451
Epoch 4/10
17377/17377 [==============================] - 7s 394us/step - loss: 21.8914 - acc: 0.1499
Epoch 5/10
17377/17377 [==============================] - 7s 391us/step - loss: 21.6447 - acc: 0.1508
Epoch 6/10
17377/17377 [==============================] - 7s 389us/step - loss: 21.4116 - acc: 0.1518
Epoch 7/10
17377/17377 [==============================] - 7s 392us/step - loss: 21.2326 - acc: 0.1564
Epoch 8/10
17377/17377 [==============================] - 7s 387us/step - loss: 21.0915 - acc: 0.1506
Epoch 9/10
17377/17377 [==============================] - 7s 390us/step - loss: 20.9502 - acc: 0.1590
Epoch 10/10
17377/17377 [==============================] - 7s 382us/st

In [159]:
calculate_all(M_word2vec_our_vect_train, M_word2vec_our_vect_test, 'Word2Vec(our) + NN', model = model)

Word2Vec(our) + NN
Epoch 1/10
17377/17377 [==============================] - 6s 371us/step - loss: 75.3157 - acc: 0.0383
Epoch 2/10
17377/17377 [==============================] - 7s 380us/step - loss: 63.5065 - acc: 0.0741
Epoch 3/10
17377/17377 [==============================] - 7s 383us/step - loss: 44.0635 - acc: 0.0921
Epoch 4/10
17377/17377 [==============================] - 7s 381us/step - loss: 33.9469 - acc: 0.1030
Epoch 5/10
17377/17377 [==============================] - 7s 392us/step - loss: 27.0573 - acc: 0.1192
Epoch 6/10
17377/17377 [==============================] - 7s 381us/step - loss: 24.3746 - acc: 0.1299
Epoch 7/10
17377/17377 [==============================] - 7s 381us/step - loss: 23.2316 - acc: 0.1358
Epoch 8/10
17377/17377 [==============================] - 7s 384us/step - loss: 22.4616 - acc: 0.1410
Epoch 9/10
17377/17377 [==============================] - 7s 390us/step - loss: 21.9847 - acc: 0.1444
Epoch 10/10
17377/17377 [==============================] - 7s 3

In [160]:
calculate_all(M_word2vec_google_vect_train, M_word2vec_google_vect_test, 'Word2Vec(google news) + NN', model=model)

Word2Vec(google news) + NN
Epoch 1/10
17377/17377 [==============================] - 7s 376us/step - loss: 27.0848 - acc: 0.0993
Epoch 2/10
17377/17377 [==============================] - 7s 390us/step - loss: 23.9112 - acc: 0.1324
Epoch 3/10
17377/17377 [==============================] - 7s 376us/step - loss: 23.0701 - acc: 0.1387
Epoch 4/10
17377/17377 [==============================] - 7s 381us/step - loss: 22.5819 - acc: 0.1399
Epoch 5/10
17377/17377 [==============================] - 7s 391us/step - loss: 22.2509 - acc: 0.1446
Epoch 6/10
17377/17377 [==============================] - 7s 388us/step - loss: 21.9594 - acc: 0.1450
Epoch 7/10
17377/17377 [==============================] - 7s 382us/step - loss: 21.7603 - acc: 0.1474
Epoch 8/10
17377/17377 [==============================] - 7s 383us/step - loss: 21.5391 - acc: 0.1469
Epoch 9/10
17377/17377 [==============================] - 7s 383us/step - loss: 21.4108 - acc: 0.1524
Epoch 10/10
17377/17377 [==============================

In [ ]:
doc2vec NN1

In [238]:
M_doc2vec_test.shape

(10229, 100)

In [240]:
%%time
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(100, input_shape=(100,)),
    Activation('relu'),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


calculate_all(M_doc2vec_train, M_doc2vec_test, 'Doc2Vec + NN1', model = model)

Doc2Vec + NN1
Epoch 1/10
17377/17377 [==============================] - 6s 341us/step - loss: 30.2344 - acc: 0.0661
Epoch 2/10
17377/17377 [==============================] - 4s 254us/step - loss: 26.5249 - acc: 0.1031
Epoch 3/10
17377/17377 [==============================] - 4s 253us/step - loss: 25.1200 - acc: 0.1165
Epoch 4/10
17377/17377 [==============================] - 4s 255us/step - loss: 24.3594 - acc: 0.1213
Epoch 5/10
17377/17377 [==============================] - 4s 253us/step - loss: 23.8925 - acc: 0.1284
Epoch 6/10
17377/17377 [==============================] - 4s 251us/step - loss: 23.5945 - acc: 0.1295
Epoch 7/10
17377/17377 [==============================] - 4s 254us/step - loss: 23.3769 - acc: 0.1331
Epoch 8/10
17377/17377 [==============================] - 5s 259us/step - loss: 23.1940 - acc: 0.1357
Epoch 9/10
17377/17377 [==============================] - 4s 256us/step - loss: 23.0534 - acc: 0.1354
Epoch 10/10
17377/17377 [==============================] - 4s 256us/

In [ ]:
doc2vec NN2

In [232]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(256, input_shape=(100,)),
    Activation('relu'),
    Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

calculate_all(M_doc2vec_train, M_doc2vec_test, 'Doc2Vec + NN2', model = model)

Word2Vec(our) + NN
Epoch 1/10
17377/17377 [==============================] - 7s 402us/step - loss: 29.2474 - acc: 0.0819
Epoch 2/10
17377/17377 [==============================] - 6s 361us/step - loss: 25.6303 - acc: 0.1196
Epoch 3/10
17377/17377 [==============================] - 6s 353us/step - loss: 24.5594 - acc: 0.1254
Epoch 4/10
17377/17377 [==============================] - 6s 362us/step - loss: 24.0201 - acc: 0.1283
Epoch 5/10
17377/17377 [==============================] - 6s 359us/step - loss: 23.6748 - acc: 0.1310
Epoch 6/10
17377/17377 [==============================] - 6s 371us/step - loss: 23.4644 - acc: 0.1333
Epoch 7/10
17377/17377 [==============================] - 6s 366us/step - loss: 23.2999 - acc: 0.1340
Epoch 8/10
17377/17377 [==============================] - 7s 380us/step - loss: 23.1776 - acc: 0.1343
Epoch 9/10
17377/17377 [==============================] - 6s 370us/step - loss: 23.0967 - acc: 0.1341
Epoch 10/10
17377/17377 [==============================] - 7s 3

doc2vec NN3

In [236]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(512, input_shape=(100,)),
    Activation('relu'),
    Dropout(0.1),
    Dense(256),
    Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

calculate_all(M_doc2vec_train, M_doc2vec_test, 'Doc2Vec + NN3', model = model)

Word2Vec(our) + NN
Epoch 1/10
17377/17377 [==============================] - 9s 519us/step - loss: 28.7775 - acc: 0.0781
Epoch 2/10
17377/17377 [==============================] - 8s 444us/step - loss: 25.4177 - acc: 0.1079
Epoch 3/10
17377/17377 [==============================] - 8s 448us/step - loss: 24.2434 - acc: 0.1229
Epoch 4/10
17377/17377 [==============================] - 8s 435us/step - loss: 23.5863 - acc: 0.1333
Epoch 5/10
17377/17377 [==============================] - 7s 430us/step - loss: 23.0725 - acc: 0.1365
Epoch 6/10
17377/17377 [==============================] - 8s 450us/step - loss: 22.7232 - acc: 0.1414
Epoch 7/10
17377/17377 [==============================] - 8s 450us/step - loss: 22.4000 - acc: 0.1425
Epoch 8/10
17377/17377 [==============================] - 7s 417us/step - loss: 22.1671 - acc: 0.1416
Epoch 9/10
17377/17377 [==============================] - 8s 432us/step - loss: 21.9976 - acc: 0.1432
Epoch 10/10
17377/17377 [==============================] - 8s 4

In [258]:
lda_dict = pickle.load(open('/mnt/dump/lda_dict.pickle', 'rb'))

In [259]:
len(lda_dict.keys())

19314

In [260]:
lda_dict[14]

array([4.21229999e-05, 4.21229997e-05, 4.21229997e-05, 4.21229998e-05,
       4.21229997e-05, 4.21229998e-05, 3.05375556e-01, 4.21229999e-05,
       1.00020002e-02, 4.21229998e-05, 6.83906353e-01, 4.21229997e-05,
       4.21230001e-05, 4.21229997e-05, 4.21229998e-05, 4.21229996e-05,
       4.21229996e-05, 4.21229996e-05, 4.21229997e-05, 4.21229999e-05])

In [254]:
df_test['label'].nunique()

1931

In [261]:
M_lda_train = np.array([lda_dict[lbl] for lbl in df_train['label']])
M_lda_test  = np.array([lda_dict[lbl] for lbl in df_test['label']])

In [262]:
M_lda_test.shape

(10229, 20)

In [263]:
model = Sequential([
    Dense(100, input_shape=(20,)),
    Activation('relu'),
#     Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


calculate_all(M_lda_train, M_lda_test, 'LDA + NN1', model = model)

Doc2Vec + NN1
Epoch 1/10
17377/17377 [==============================] - 5s 274us/step - loss: 32.8940 - acc: 0.0156
Epoch 2/10
17377/17377 [==============================] - 4s 251us/step - loss: 30.4740 - acc: 0.0265
Epoch 3/10
17377/17377 [==============================] - 4s 252us/step - loss: 29.3486 - acc: 0.0482
Epoch 4/10
17377/17377 [==============================] - 4s 252us/step - loss: 28.7107 - acc: 0.0628
Epoch 5/10
17377/17377 [==============================] - 4s 251us/step - loss: 28.3009 - acc: 0.0697
Epoch 6/10
17377/17377 [==============================] - 4s 252us/step - loss: 28.0051 - acc: 0.0734
Epoch 7/10
17377/17377 [==============================] - 4s 252us/step - loss: 27.7738 - acc: 0.0720
Epoch 8/10
17377/17377 [==============================] - 4s 250us/step - loss: 27.5841 - acc: 0.0751
Epoch 9/10
17377/17377 [==============================] - 4s 249us/step - loss: 27.4284 - acc: 0.0737
Epoch 10/10
17377/17377 [==============================] - 4s 250us/

In [266]:
%%time
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential([
    Dense(256, input_shape=(20,)),
    Activation('relu'),
    Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

calculate_all(M_lda_train, M_lda_test, 'LDA + NN2', model = model)

LDA + NN2
Epoch 1/10
17377/17377 [==============================] - 8s 482us/step - loss: 32.1313 - acc: 0.0201
Epoch 2/10
17377/17377 [==============================] - 7s 424us/step - loss: 29.6327 - acc: 0.0444
Epoch 3/10
17377/17377 [==============================] - 7s 413us/step - loss: 28.7118 - acc: 0.0659
Epoch 4/10
17377/17377 [==============================] - 7s 423us/step - loss: 28.1926 - acc: 0.0727
Epoch 5/10
17377/17377 [==============================] - 8s 441us/step - loss: 27.8475 - acc: 0.0733
Epoch 6/10
17377/17377 [==============================] - 8s 442us/step - loss: 27.5872 - acc: 0.0750
Epoch 7/10
17377/17377 [==============================] - 7s 416us/step - loss: 27.3887 - acc: 0.0718
Epoch 8/10
17377/17377 [==============================] - 8s 433us/step - loss: 27.2329 - acc: 0.0702
Epoch 9/10
17377/17377 [==============================] - 7s 426us/step - loss: 27.0976 - acc: 0.0695
Epoch 10/10
17377/17377 [==============================] - 8s 434us/step

In [267]:
model = Sequential([
    Dense(512, input_shape=(20,)),
    Activation('relu'),
    Dropout(0.1),
    Dense(256),
    Dropout(0.1),
    Dense(2007),
    Activation('softmax'),
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

calculate_all(M_lda_train, M_lda_test, 'LDA + NN3', model = model)

LDA + NN3
Epoch 1/10
17377/17377 [==============================] - 8s 463us/step - loss: 30.9110 - acc: 0.0353
Epoch 2/10
17377/17377 [==============================] - 9s 533us/step - loss: 29.0405 - acc: 0.0577
Epoch 3/10
17377/17377 [==============================] - 7s 414us/step - loss: 28.6817 - acc: 0.0638
Epoch 4/10
17377/17377 [==============================] - 7s 412us/step - loss: 28.5128 - acc: 0.0667
Epoch 5/10
17377/17377 [==============================] - 7s 410us/step - loss: 28.3475 - acc: 0.0687
Epoch 6/10
17377/17377 [==============================] - 7s 419us/step - loss: 28.2225 - acc: 0.0727
Epoch 7/10
17377/17377 [==============================] - 7s 420us/step - loss: 28.0561 - acc: 0.0696
Epoch 8/10
17377/17377 [==============================] - 8s 481us/step - loss: 27.9555 - acc: 0.0730
Epoch 9/10
17377/17377 [==============================] - 8s 467us/step - loss: 27.8181 - acc: 0.0760
Epoch 10/10
17377/17377 [==============================] - 7s 416us/step

In [74]:
df_test.groupby('label')

In [69]:
max(df_train['title'])

2006

In [270]:
txt = """ROC  0.7546968998209596
AP   0.3075651931425528
ROC  0.9999799418568306
AP   0.9987357606495607
ROC  0.7532594150112648
AP   0.297674106771554
ROC  0.9999616781827476
AP   0.9986121067475677
ROC  0.7469967012596413
AP   0.27351119401116275
ROC  0.9999783476477072
AP   0.9993915716941449
ROC  0.7179403852652685
AP   0.22118009257703494
ROC  0.999981500431548
AP   0.9992350922300626
ROC:  0.896871795675903
AP:  0.303766823903395
ROC:  0.9540384128291943
AP:  0.4196979823178109
ROC:  0.9286447493883703
AP:  0.38491135823766903
ROC:  0.9680995098492765
AP:  0.4438656621749801
ROC:  0.9423935675580685
AP:  0.4328189990064034
ROC:  0.981399795628338
AP:  0.5444310838687031
ROC:  0.9326893930202086
AP:  0.3937610837903299
ROC:  0.9706262656156158
AP:  0.4571574749818919
ROC:  0.9085013561204631
AP:  0.35226382183638516
ROC:  0.9681486465101421
AP:  0.48654587563221113
ROC:  0.9345715045108258
AP:  0.4223460399281312
ROC:  0.9757282070122232
AP:  0.5072046921403474
ROC:  0.9424305346162258
AP:  0.4566663669574004
ROC:  0.9837583592597311
AP:  0.5931163931260031
ROC:  0.9366086480470165
AP:  0.42468410225164505
ROC:  0.9752311375100895
AP:  0.5045628384530649
ROC:  0.9071444931225039
AP:  0.4161448243644264
ROC:  0.9738340362566488
AP:  0.6236025128880227
ROC:  0.9075679308768607
AP:  0.43326529111824
ROC:  0.9647508476492808
AP:  0.5606445264025705
ROC:  0.9066859515179184
AP:  0.4232705732371311
ROC:  0.9094973823582159
AP:  0.42325137835017673
ROC:  0.9094973823582159
AP:  0.42325137835017673
ROC:  0.9094973823582159
AP:  0.42325137835017673
ROC:  0.9103335471891381
AP:  0.36798968377681257
ROC:  0.957744973189078
AP:  0.4593654851204441
ROC:  0.9101419966814267
AP:  0.37537264381318514
ROC:  0.9580369747060608
AP:  0.4844547291179492
ROC:  0.9077998059094872
AP:  0.34269591438484587
ROC:  0.9630838017820144
AP:  0.4877391588760884
ROC:  0.8845217770914648
AP:  0.20170151524521007
ROC:  0.9290241365384478
AP:  0.22329419024499023
ROC:  0.8880336284957404
AP:  0.2161155100610153
ROC:  0.9335511568741998
AP:  0.2409500331342773
ROC:  0.8747194390674969
AP:  0.21764204821061797
ROC:  0.8747194390674969
AP:  0.21764204821061797"""

In [275]:
AP_train = []
AP_test  = []
R_train =  []
R_test  = []
for c, line in enumerate(txt.split('\n')):
    f = round(float(line.split('  ')[-1]), 3)
    if c%4==0:
        R_test.append(f)
        
    if c%4==1:
        AP_test.append(f)
        
    if c%4==2:
        R_train.append(f)
        
    if c%4==3:
        AP_train.append(f)

In [281]:
print('\n'.join([str(i) for i in AP_train]))

0.999
0.999
0.999
0.999
0.42
0.444
0.544
0.457
0.487
0.507
0.593
0.505
0.624
0.561
0.423
0.423
0.459
0.484
0.488
0.223
0.241
0.218


In [282]:
len(AP_train)

22

In [ ]:
0.755
0.753
0.747
0.718
0.897
0.929
0.942
0.933
0.909
0.935
0.942
0.937
0.907
0.908
0.907
0.909
0.91
0.91
0.908
0.885
0.888
0.875

In [126]:
# data_train.head()

In [128]:
# data_test.head()

In [129]:
# data_train.to_excel(train_path)
# data_test.to_excel(test_path)

Все темы есть и в train и в test

In [130]:
# np.mean(data_train['title'].unique() == data_test['title'].unique())

In [131]:
# print('Всего классов ', data_test['title'].nunique())

Все labels не пересекаются

In [132]:
# set(data_test['label'].unique()) & set(data_train['label'].unique())

In [89]:
# data.head()

In [90]:
# data = pd.read_excel('/mnt/dump/data.xlsx')
# data = data[['label', 'text_preprocess_with_out_stop_words', 'title']]
# data = data.rename(columns={'text_preprocess_with_out_stop_words': 'text'})

In [91]:
# data.head()

In [92]:
# data.to_excel('/mnt/dump/df.xlsx')

In [93]:
# data.shape